In [1]:
import re
import shutil
import pandas as pd
from pandas import DataFrame, Series
import github3
from github3 import login
from git import Repo, GitCommandError
import yaml
from subprocess import Popen, PIPE, STDOUT, SubprocessError, check_output
from configparser import ConfigParser
from github3.repos.branch import Branch
from github3.null import NullObject
from schematics.models import Model, BaseType
from schematics.types import StringType, IntType, BooleanType
from schematics.types.compound import ListType, DictType
from schematics.exceptions import ValidationError
%cd ~/Documents/projects/stitch/python/
from stitch.core.stitch_frame import StitchFrame
%cd ~/Documents/projects/nerve/python/
from nerve.core.model import Nerve
from nerve.core.client import Client
from nerve.core.utils import *
from nerve.core.git import Git
from nerve.core.git_lfs import GitLFS
from nerve.core.metadata import Metadata
from nerve.core.utils import execute_subprocess, get_status
from nerve.spec.specifications import *
from nerve.spec.validators import *
from nerve.spec.traits import *
from nerve.spec import traits
from nerve.test.test_suite import *
%cd ~/Documents/projects/nerve

/Users/alex/Documents/projects/stitch/python
/Users/alex/Documents/projects/nerve/python
/Users/alex/Documents/projects/nerve


In [2]:
config_path = '/Users/alex/Documents/projects/nerve/python/nerve/resources/nerverc.yml'
nerve = Nerve(config_path)
proj = os.path.join(nerve['project-root'], nerve['project']['project-name'])
info = nerve._get_info(None, None, {})
cc = info.client_conf

In [5]:
nerve.delete(True, True)

True

In [15]:
nerve.delete(False, True)

True

In [6]:
nerve.create(notes='some note about a thing')

True

In [7]:
nerve.clone()

True

In [8]:
%%bash
cd /tmp/ntr001
echo vasc v001 > maya001/ntr001_maya001_vasc_v001.mb
echo kidney v001 > maya001/ntr001_maya001_kdny_v001.mb
tree

.
├── geo001
├── maya001
│   ├── ntr001_maya001_kdny_v001.mb
│   └── ntr001_maya001_vasc_v001.mb
├── proj001_meta.yml
└── vol001

3 directories, 3 files


In [27]:
d = DataFrame(list(stat)).applymap(lambda x: x.data)
st = StitchFrame(d).flatten(prefix=False)
st.to_dataframe().T

,0,1
asset-id,dc09e946-fd04-485e-a916-7f...,0529141b-0afd-4271-a5ff-be...
asset-name,ntr001_maya001_kdny_v001,ntr001_maya001_vasc_v001
asset-type,nondeliverable,nondeliverable
data,None,None
notes,,
project-id,None,None
project-name,ntr001,ntr001
specification,maya001,maya001
url,None,None


In [28]:
nerve.publish(notes='create vol001_v001; if you can read this, it probably worked', verbosity=2)

UnprocessableEntity: 422 Validation Failed

In [5]:
# local = Git(info.path, environment=info.env)
# DataFrame(list(local.references()))

In [12]:
self = nerve
config = info.config
branch = info.branch

# pulling metadata first avoids merge conflicts by keeping the
# user-branch HEAD ahead of the dev branch
local.branch('dev')
local.pull('dev')
local.merge('dev', branch)
local.branch(branch)

In [14]:
# get nondeliverable assets
nondeliverables = self.status(status_asset_types=['nondeliverable'], **config)
nondeliverables = list(nondeliverables)
for non in nondeliverables:
    non.get_traits()
    non.write(validate=False)

# publish nondeliverables
if len(nondeliverables) > 0:
    # push non-deliverables to user-branch
    local.add([x.metapath for x in nondeliverables])
    local.add([x.datapath for x in nondeliverables])
    names = [x['asset-name'] for x in nondeliverables]
    local.commit('NON-DELIVERABLES: ' + ', '.join(names))

In [18]:
# lfs = GitLFS(proj, info.env)
# lfs.prepush(local.url)
local.push(info.branch)

GitCommandError: 'git push --porcelain origin alex' returned with exit code 1